In [ ]:
from braket.circuits import Circuit
import itertools as it
import random

def cnot_sequence(circuit_length, input_bits='00'):
    c = Circuit()
    
    # encode the input
    if input_bits[0] == '1': c.x(0)
    if input_bits[1] == '1': c.x(1)
        
    # create the CNOT sequence
    for idx in range(circuit_length):
        if idx % 2 == 0: c.cnot(0, 1)
        if idx % 2 == 1: c.cnot(1, 0)
            
    return c

possible_input_bits      = ('00', '01', '10', '11')
possible_circuit_lengths = (1, 2, 3, 4, 5, 6)
possible_keys = list(it.product(possible_input_bits, possible_circuit_lengths))

circuit_pool             = {}
for input_bits in possible_input_bits:
    for circuit_length in possible_circuit_lengths:
        circuit = cnot_sequence(circuit_length, input_bits=input_bits)
        circuit_pool[(input_bits, circuit_length)] = circuit

def generate_circuit_batch(size):
    random_keys = random.choices(possible_keys, k=size)
    return [(key, circuit_pool[key]) for key in random_keys]

print("EXAMPLE CIRCUIT BATCH")
print("_" * 24)
for key, circuit in generate_circuit_batch(5):
    print()
    print("input_bits={}, length={}".format(*key))
    print("-" * 24)
    print(circuit)
    print("_" * 24)

In [ ]:
# The following is just a convenient little timer for things that take a while in the code below.
# I snipped it from https://medium.com/@DahlitzF/how-to-create-your-own-timing-context-manager-in-python-a0e944b48cf8

from time import time

class Timer:
    def __init__(self, description):
        self.description = description
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        print(f"complete: {self.end - self.start:.3f}")

In [ ]:
# On with the actual code...
        
from braket.aws import AwsDevice

aws_simulator = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
ionq = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
s3_folder = ("amazon-braket-quasimodo-840eb696707f", "yuval_mwe_preliminary")

reference_table = {}

for input_bits in possible_input_bits:
    for circuit_length in possible_circuit_lengths:
        
        circuit_to_simulate = circuit_pool[(input_bits, circuit_length)]
        print(f"Starting job: input_bits={input_bits}, circuit_length={circuit_length}...", end=" ")
        
        with Timer("AWS simulator"):
            # First create the task for Amazon's circuit simulator.
            # Note that the number of shots is irrelevant here;
            # I want the simulated measurement probabilities only.
            task = aws_simulator.run(circuit_to_simulate, s3_folder, shots=1)
            
            # Now I can extract the measurement probabilities.
            # This is a dictionary of the form {correct_output: 1}.
            # The useful information is therefore the key, not the value.
            result = task.result().measurement_probabilities
            correct_output = list(result.keys())[0]
            
            # I now store the correct output in the reference table.
            reference_table[(input_bits, circuit_length)] = correct_output

In [ ]:
for input_bits in possible_input_bits:
    reference = [input_bits] + [reference_table[(input_bits, l)] for l in possible_circuit_lengths]
    print("{} | {} {} {} {} {} {}".format(*reference))

In [ ]:
test_batch = generate_circuit_batch(1000)
tasks = [ionq.run(circuit, s3_folder, shots=1000) for key, circuit in test_batch]
task_id = [t.id for t in tasks]
print(task_id)

In [ ]:
state_list = []

with Timer("Fetching the state takes super long..."):
    for k in range(len(tasks)):
        state = tasks[k].state()
        state_list += [state]
        print(f"Task {k:>3}: {state:<10}", end="\r")
    print("State fetch", end=" ")

number_of_tasks_still_queued = sum(1 if state_list[k] == 'QUEUED' else 0
                                   for k in range(len(state_list)))

if number_of_tasks_still_queued == 0:
    print("All tasks complete!")
else:
    print(f"Still waiting on {number_of_tasks_still_queued} tasks.")

In [ ]:
mport pandas as pd
import matplotlib.pyplot as plt

data = []

for idx in range(len(tasks)):
    the_task   = tasks[idx]
    task_id    = the_task.id
    the_result = the_task.result()
    
    start_time   = pd.to_datetime(the_result.task_metadata.createdAt)
    end_time     = pd.to_datetime(the_result.task_metadata.endedAt)
    elapsed_time = end_time - start_time
    
    batch_member   = test_batch[idx]
    circuit_key    = batch_member[0]
    input_bits     = circuit_key[0]
    circuit_length = circuit_key[1]
    
    measurement_probabilities = the_result.measurement_probabilities
    correct_output            = reference_table[circuit_key]
    success_probability       = measurement_probabilities[correct_output]
    
    data.append((task_id,
                 input_bits,
                 circuit_length,
                 start_time,
                 end_time,
                 elapsed_time,
                 success_probability))

data = pd.DataFrame(data, columns=['id_string',
                                   'input_bits',
                                   'circuit_length',
                                   'start_time',
                                   'end_time',
                                   'elapsed_time',
                                   'success_probability'])

In [ ]:
data['elapsed_time'].astype('timedelta64[h]').hist()
plt.xlabel("Number of hours")
plt.ylabel("Number of tasks")
plt.title("Amount of time waited to obtain data from IonQ")
plt.show()

In [ ]:
data['success_probability'].hist()
plt.xlabel("Probability of success")
plt.ylabel("Number of circuits")
plt.title("A simple histogram of the circuit success probabilities")
plt.show()

In [ ]:
plt.hist([df["success_probability"] for input_bits, df in data.groupby("input_bits")],
          bins=50, range=(0.7, 1.0), stacked=True)

plt.xlabel("Probability of success")
plt.ylabel("Number of circuits")
plt.title("Circuit success probabilities grouped by input")
plt.show()

In [ ]:
plt.hist([df["success_probability"] for input_bits, df in data.groupby("circuit_length")],
          bins=50, range=(0.7, 1.0), stacked=True)

plt.xlabel("Probability of success")
plt.ylabel("Number of circuits")
plt.title("Circuit success probabilities grouped by circuit length")
plt.show()

In [ ]:
data.plot.scatter(x="end_time", y="success_probability")
plt.xlabel("Circuit execution timestamp")
plt.ylabel("Success probability")
plt.title("Variable quality of execution windows")
plt.show()

In [ ]:
data.plot.scatter(x="end_time", y="success_probability", c='circuit_length', marker='.')
plt.xlabel("Circuit execution timestamp")
plt.ylabel("Success probability")
plt.title("Execution window data grouped by circuit length")
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.violinplot(data=data, x='circuit_length', y='success_probability')
plt.xlabel("Circuit Length")
plt.ylabel("Success Probability")
plt.title("Performance for random CNOT test")
plt.show()

In [ ]:
data['good?'] = data['success_probability'] > (90 + (7 - data['circuit_length']))/100
sns.violinplot(data=data, x='circuit_length', y='success_probability', hue='good?')
plt.xlabel("Circuit Length")
plt.ylabel("Success Probability")
plt.title("Performance for random CNOT test")
plt.show()

In [ ]:
sns.kdeplot(data=data, x='end_time', hue='good?', multiple='fill')
plt.xlabel("Circuit Execution Timestamp")
plt.xticks(rotation=-90)
plt.title("Simple Execution Window Classification")
plt.show()

In [ ]:
data['execution_window'] = 'discarded'

end_A   = pd.Timestamp(year=2023, month=1, day=29, hour=22, tz='UTC')
start_B = pd.Timestamp(year=2023, month=1, day=30, hour=2, tz='UTC')
end_B   = pd.Timestamp(year=2023, month=1, day=30, hour=4, tz='UTC')

data.loc[data['end_time'] < end_A, 'execution_window'] = 'A'
data.loc[(data['end_time'] >= start_B) & (data['end_time'] < end_B), 'execution_window'] = 'B'

sns.lmplot(data=data, x='circuit_length', y='success_probability', hue='execution_window')
plt.xlabel("Circuit Length")
plt.ylabel("Success Probability")
plt.title("Performance classified by execution window")
plt.show()

In [ ]:
g = sns.kdeplot(data=data.groupby('circuit_length').get_group(6),
                x='success_probability', hue='execution_window', multiple='fill')
sns.move_legend(g, 'upper left', title='Execution Window')
plt.xlabel('Success Probability')
plt.xlim(0.7, 1.0)
plt.title("Execution window quality check")
plt.show()

In [ ]:
sns.histplot(data=data, x='end_time', bins=100)
plt.xticks(rotation=-90)
plt.xlabel("Execution timestamp")
plt.title("Distribution of execution times for a set of 1000 circuits")
plt.show()

In [ ]:
for id_str in task_id:
    print(id_str)

In [ ]:
for key, circuit in test_batch:
    print(key)

In [ ]:
len(tasks)